In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [3]:
from keras.models import load_model

2022-12-19 17:09:10.233543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd

### Get weights from pre-trained Keras model

In [4]:
keras_DNN = load_model("CF_Mortality.model")

2022-12-19 17:02:50.492444: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
keras_DNN.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128)]             0         
                                                                 
 dropout_21 (Dropout)        (None, 128)               0         
                                                                 
 dense_25 (Dense)            (None, 128)               16512     
                                                                 
 dropout_22 (Dropout)        (None, 128)               0         
                                                                 
 dense_26 (Dense)            (None, 64)                8256      
                                                                 
 dropout_23 (Dropout)        (None, 64)                0         
                                                                 
 dense_27 (Dense)            (None, 64)                4160

In [12]:
keras_DNN.layers[-2].output

<KerasTensor: shape=(None, 16) dtype=float32 (created by layer 'dense_29')>

In [6]:
keras_DNN_weights = keras_DNN.get_weights()

### Reconstruct Keras model in PyTorch


In [ ]:
'''
def DNN_Simple(shape):
    X_in = Input(shape=(shape,))
    H2 = Dense(128, activation='relu', kernel_regularizer=l2(5e-4))(X_in)
    H3 = Dropout(0.5)(H2)
    H4 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H3)
    H5 = Dropout(0.2)(H4)
    H6 = Dense(64, activation='relu', kernel_regularizer=l2(5e-4))(H5)
    H7 = Dropout(0.2)(H6)
    H8 = Dense(48, activation='relu', kernel_regularizer=l2(5e-4))(H7)
    H9 = Dropout(0.2)(H8)
    H10 = Dense(16, activation='relu', kernel_regularizer=l2(5e-4))(H9)
    H11 = Dropout(0.5)(H10)
    Y = Dense(3, activation='softmax')(H11)
    model = Model(inputs=X_in, outputs=Y)
    model.compile(optimizer=optimizers.Adam(decay=0.01),loss='categorical_crossentropy'
                  ,metrics=['accuracy']) 
    return model
'''

In [8]:
class DNN(nn.Module):
    def __init__(self, input_shape):
        super(DNN, self).__init__()
        self.fc1 = nn.Linear(input_shape, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 48)
        self.fc5 = nn.Linear(48, 16)
#         self.output = nn.Linear(16, 2)
        
        self.dropout_50 = nn.Dropout(0.5)
        self.dropout_20 = nn.Dropout(0.2)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout_50(x)
        x = F.relu(self.fc2(x))
        x = self.dropout_20(x)
        x = F.relu(self.fc3(x))
        x = self.dropout_20(x)
        x = F.relu(self.fc4(x))
        x = self.dropout_20(x)
#         x = F.relu(self.fc5(x))
        return self.fc5(x)
#         x = self.dropout_50(x)
#         return F.softmax(self.output(x))

### Transfer pre-trained weights from Keras model to PyTorch

In [10]:
pytorch_DNN = DNN(128)

In [11]:
pytorch_DNN

DNN(
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=48, bias=True)
  (fc5): Linear(in_features=48, out_features=16, bias=True)
  (dropout_50): Dropout(p=0.5, inplace=False)
  (dropout_20): Dropout(p=0.2, inplace=False)
)

In [12]:
pytorch_DNN.fc1.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[0]))
pytorch_DNN.fc1.bias.data = torch.from_numpy(keras_DNN_weights[1])
pytorch_DNN.fc2.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[2]))
pytorch_DNN.fc2.bias.data = torch.from_numpy(keras_DNN_weights[3])
pytorch_DNN.fc3.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[4]))
pytorch_DNN.fc3.bias.data = torch.from_numpy(keras_DNN_weights[5])
pytorch_DNN.fc4.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[6]))
pytorch_DNN.fc4.bias.data = torch.from_numpy(keras_DNN_weights[7])
pytorch_DNN.fc5.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[8]))
pytorch_DNN.fc5.bias.data = torch.from_numpy(keras_DNN_weights[9])
# pytorch_DNN.output.weight.data = torch.from_numpy(np.transpose(keras_DNN_weights[10]))
# pytorch_DNN.output.bias.data = torch.from_numpy(keras_DNN_weights[11])

### Load data

In [7]:
data_df = pd.read_csv("matt_metadata_norm.csv", sep='\t')

In [8]:
data_df.head()

,Patient,Age,Gender,Body temperature,Underlying diseases,MCHC,MCH,MCV,HCT,HGB,...,FDG,LPS,U,UALB,BCF8,ASO,PS,RF,PC,LAC
0,Patient 1,81.0,0,0.983871,1,0.736842,0.600000,0.483333,-0.42,-0.155556,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
1,Patient 2,50.0,0,1.040323,1,0.868421,0.642857,0.455556,0.15,0.333333,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
2,Patient 3,65.0,1,1.034946,1,0.368421,0.585714,0.677778,-0.96,-0.644444,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
3,Patient 4,73.0,0,1.034946,1,0.552632,0.528571,0.516667,0.09,0.177778,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
4,Patient 5,64.0,1,1.021505,1,0.342105,0.485714,0.577778,0.15,0.133333,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5


In [9]:
data_df.drop(["Patient"], axis=1, inplace=True)

In [10]:
# Our data has 127 features but the pre-trained model accepts 128 features
# We can add 1 column filled with 0.5s to make the data also have 128 features
# However, we don't know whether we should append this columns at the end of the data or in the beginning
# because we don't know how the pre-trained model accepted the original input

In [11]:
data_df_beg = data_df.copy(deep=True)
data_df_beg.insert(loc=0, column="addition", value=[0.5 for i in range(len(data_df))])
data_df_end = data_df.copy(deep=True)
data_df_end["addition"] = [0.5 for i in range(len(data_df))]

In [12]:
data_df_beg.head()

,addition,Age,Gender,Body temperature,Underlying diseases,MCHC,MCH,MCV,HCT,HGB,...,FDG,LPS,U,UALB,BCF8,ASO,PS,RF,PC,LAC
0,0.5,81.0,0,0.983871,1,0.736842,0.600000,0.483333,-0.42,-0.155556,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
1,0.5,50.0,0,1.040323,1,0.868421,0.642857,0.455556,0.15,0.333333,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
2,0.5,65.0,1,1.034946,1,0.368421,0.585714,0.677778,-0.96,-0.644444,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
3,0.5,73.0,0,1.034946,1,0.552632,0.528571,0.516667,0.09,0.177778,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5
4,0.5,64.0,1,1.021505,1,0.342105,0.485714,0.577778,0.15,0.133333,...,0.5,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5


In [13]:
data_df_end.head()

,Age,Gender,Body temperature,Underlying diseases,MCHC,MCH,MCV,HCT,HGB,RBC,...,LPS,U,UALB,BCF8,ASO,PS,RF,PC,LAC,addition
0,81.0,0,0.983871,1,0.736842,0.600000,0.483333,-0.42,-0.155556,-0.240000,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,0.5
1,50.0,0,1.040323,1,0.868421,0.642857,0.455556,0.15,0.333333,0.200000,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,0.5
2,65.0,1,1.034946,1,0.368421,0.585714,0.677778,-0.96,-0.644444,-0.713333,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,0.5
3,73.0,0,1.034946,1,0.552632,0.528571,0.516667,0.09,0.177778,0.120000,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,0.5
4,64.0,1,1.021505,1,0.342105,0.485714,0.577778,0.15,0.133333,0.126667,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,0.5


In [20]:
X_beg = torch.Tensor(data_df_beg.values)
X_end = torch.Tensor(data_df_end.values)

In [21]:
print("X_beg shape:", X_beg.shape)
print("X_end shape:", X_end.shape)

X_beg shape: torch.Size([1521, 128])
X_end shape: torch.Size([1521, 128])


In [22]:
X_beg

tensor([[ 0.5000, 81.0000,  0.0000,  ...,  0.5000, -0.9929,  0.5000],
        [ 0.5000, 50.0000,  0.0000,  ...,  0.5000, -0.9929,  0.5000],
        [ 0.5000, 65.0000,  1.0000,  ...,  0.5000, -0.9929,  0.5000],
        ...,
        [ 0.5000, 71.0000,  1.0000,  ...,  0.5000, -0.9929,  0.5000],
        [ 0.5000, 66.0000,  1.0000,  ...,  0.5000, -0.9929,  0.5000],
        [ 0.5000, 64.0000,  1.0000,  ...,  0.5000, -0.9929,  0.5000]])

In [23]:
X_beg_numpy = X_beg.numpy()
X_end_numpy = X_end.numpy()

In [24]:
X_beg_numpy

array([[ 0.5       , 81.        ,  0.        , ...,  0.5       ,
        -0.99285716,  0.5       ],
       [ 0.5       , 50.        ,  0.        , ...,  0.5       ,
        -0.99285716,  0.5       ],
       [ 0.5       , 65.        ,  1.        , ...,  0.5       ,
        -0.99285716,  0.5       ],
       ...,
       [ 0.5       , 71.        ,  1.        , ...,  0.5       ,
        -0.99285716,  0.5       ],
       [ 0.5       , 66.        ,  1.        , ...,  0.5       ,
        -0.99285716,  0.5       ],
       [ 0.5       , 64.        ,  1.        , ...,  0.5       ,
        -0.99285716,  0.5       ]], dtype=float32)

### Feedforward

In [38]:
out_beg = pytorch_DNN(X_beg)

In [40]:
out_beg.shape

torch.Size([1521, 16])

In [41]:
out_end = pytorch_DNN(X_end)

In [42]:
out_end.shape

torch.Size([1521, 16])

### Extract tabular embeddings for morbidity

In [63]:
data_df = pd.read_csv("matt_metadata_norm_morbidity_filtered.csv", sep='\t')

In [64]:
data_df

,Patient,Age,Gender,Body temperature,Underlying diseases,MCHC,MCH,MCV,HCT,HGB,...,LPS,U,UALB,BCF8,ASO,PS,RF,PC,LAC,Morbidity
0,Patient 1,81.0,0,0.983871,1,0.736842,0.600000,0.483333,-0.42,-0.155556,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1
1,Patient 2,50.0,0,1.040323,1,0.868421,0.642857,0.455556,0.15,0.333333,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1
2,Patient 3,65.0,1,1.034946,1,0.368421,0.585714,0.677778,-0.96,-0.644444,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1
3,Patient 4,73.0,0,1.034946,1,0.552632,0.528571,0.516667,0.09,0.177778,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,2
4,Patient 5,64.0,1,1.021505,1,0.342105,0.485714,0.577778,0.15,0.133333,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,Patient 1517,45.0,1,1.045699,1,0.578947,0.442857,0.394444,-0.42,-0.200000,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1
1077,Patient 1518,66.0,1,0.983871,1,0.421053,0.628571,0.694444,-0.54,-0.333333,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1
1078,Patient 1519,71.0,1,0.983871,1,0.605263,0.871429,0.866667,-0.65,-0.355556,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1
1079,Patient 1520,66.0,1,0.983871,1,0.210526,0.457143,0.616667,-0.55,-0.400000,...,0.5,0.5,0.5,-0.86875,0.5,-0.991667,0.5,-0.992857,0.5,1


In [65]:
data_df.drop(["Patient", "Morbidity"], axis=1, inplace=True)

In [66]:
data_df.shape

(1081, 127)

In [67]:
data_df_beg = data_df.copy(deep=True)
data_df_beg.insert(loc=0, column="addition", value=[0.5 for i in range(len(data_df))])
data_df_end = data_df.copy(deep=True)
data_df_end["addition"] = [0.5 for i in range(len(data_df))]

In [68]:
X_beg_npy = data_df_beg.values
X_end_npy = data_df_end.values

In [69]:
from dnn import get_tabular_keras_dnn

In [70]:
keras_pretrained_model = get_tabular_keras_dnn()

In [71]:
pred_beg = keras_pretrained_model.predict(X_beg_npy)

34/34 [==============================] - 0s 1ms/step


In [72]:
pred_end = keras_pretrained_model.predict(X_end_npy)

34/34 [==============================] - 0s 1ms/step


In [73]:
pred_beg.shape

(1081, 16)

In [74]:
pred_end.shape

(1081, 16)

In [75]:
np.save("tabular_embeddings_morbidity_beg_filtered.npy", pred_beg)

In [76]:
np.save("tabular_embeddings_morbidity_end_filtered.npy", pred_end)

In [60]:
## Random
image_embed_morbidity_test = np.random.rand(1222, 10000)

In [62]:
np.save("image_embeddings_morbidity_test.npy", image_embed_morbidity_test)